✨This Jupyter notebook allows you to run Python code interactively 🐍✨

## Getting Started
1. Click the `Select Kernel` button at the top right.
2. Choose Python environments and select `Python 3.11.9`.

Run each section of the notebook by clicking the play button on the left side of the code cells.

## Learning Outcomes
We will focus on four key outcomes:

1. Understanding agents and prompt engineering with Prompty.
2. Utilizing Prompty tracing for debugging and observability.
3. Building and running Contoso Creative Writer.
4. Setting up automated evaluations with GitHub Actions.

Let’s start with the first one.

## 1. Understanding Agents and Prompt Engineering with Prompty
### 1.1. What are AI agents?
Contoso Creative Writer is an Agentic Application. 

**In AI, an agent is a program designed to:**

- Perceive its environment
- Make decisions
- Take actions to achieve specific goals

For Contoso Creative Writer, the goal is to help the marketing team at Contoso Outdoors write well-researched, product-specific articles. 

Contoso Creative Writer is made up of 4 agents that help achieve this goal.
<br>In the file explorer to the left open and explore the `src/api/agents` folder. This folder contains 4 sub-folders, one for each agent.  

<img src="agents.png" alt="Agents in Contoso Creative Writer" width="900" height="380">

### 1.2. Understanding how to build an agent with Prompty and Azure OpenAI

**Each agent is built with generally the same 3 files:**
 
1. `functions.json`: Describes any tools/functions available to the agent. (Some agents don't need tools besides the LLM and will not need this file.)

2. `agentname.py`: A file where the tools are written with code and user input, a prompty file and any other information are passed to the LLM. 

3. `agentname.prompty`: A file where the agents base prompt is written and LLM configurations are defined. 

**The Researcher agent:** 

To help us understand how an agent is built we will focus on the Researcher Agent.
<br>To see it in action, click the play button to the left of the cell below.

Try changing the instructions to research a topic you’re interested in, for example, “Can you find the best educational material for learning about AI Agents?”

In [ ]:
import sys
import os

# Add the path to sys.path
sys.path.append(os.path.abspath('../../src/api/agents/researcher'))

from researcher import research

instructions = "Can you find the best educational material for learning Python programming."

research(instructions=instructions)

As expected for agents built in this project, the researcher folder contains the 3 files described in the image below. 
<br>Open `src/api/agents/researcher` folder in the file explorer and click on each file to examine it. 

<img src="researcher_agent_files.png" alt="Files in researcher agent" width="900" height="480">

Now that we understand how the researcher agent works, let's see how we can edit the user instructions to influence which tools the researcher agent uses.

### 1.3. An introduction to prompt engineering with user instructions 

Prompt engineering is the process of designing and refining input prompts to guide generative AI models. 

In our earlier example you likely had your results returned from the **find_information** function which returns its results in the `web` category, 
<br>but you'll notice the `entities` and `news` categories are empty since their associated functions weren't called. 
<br>We can edit the instructions to guide the LLM to use a specific function! 

#### Find entities: 
This function is used to find information about people or places. Let's ask the LLM to find information about Guido van Rossum the creator of Python. 

In [ ]:
instructions = "Can you tell me information about the person Guido van Rossum?"

research(instructions=instructions)

#### Find news: 
This function is used to find news. Let's ask the LLM to find the latest news about Microsoft

In [ ]:
instructions = "Can you find the latest news about Microsoft? "

research(instructions=instructions)

When working with LLM's it's important to see that being explicit in your requests will return better results. 
<br>In the above examples mentioning the phrase 'person' or 'news' influenced which functions were called. 


### 1.4. Building a custom social media agent and editing the base prompt

<img src="socialmediaagent.png" alt="social media agent" width="600" height="350">

Now that we know how an agent is generally created, let's build a custom one for ourselves! 

You should see a **socialmedia** folder in the workshop folder. This folder contains:

- **A `functions.json` and `researcher.prompty` file:** We'll use these to keep the researcher agent going so we can get information from the web. 

- A new `social.py` file: This is similar to researcher.py file but adds in the `execute_social_media_writer_prompty` and `run_social_media_agent` functions. 
- **A new `social.py` file:** This is similar to researcher.py file but adds in an `execute_social_prompty` and `run_social_media_agent` function. 

- **A new `social.prompty` file:** This contains the base prompt for the social media agent.

To run this agent we will need to provide research instructions for which topics to search for online and instructions on what sort of social media content you want. 
<br>Run the cell below to test it out! 

In [ ]:
# Add the path to sys.path
sys.path.append(os.path.abspath('../../docs/workshop/socialmedia'))

from social import run_social_media_agent

research_instructions = "Find information about AI Agents"
social_media_instructions = "Write a fun and engaging twitter thread about AI Agents given the research."

run_social_media_agent(instructions=research_instructions, social_media_instructions = social_media_instructions)

 **Your task in this section is to edit the `social.prompty` file to:**
-  generate content for another social media site like LinkedIn
-  include url links in the response. 

Currently the agent is set to generate twitter content and does not include links in the returned output.
<br>Look through the researcher.prompty file if you need inspiration and remember to be as explicit as possible! 

## 2. Utilizing Prompty tracing for debugging and observabilty

When running Applications driven by LLMs, sometimes things don't go as expected! It's important to have a way to debug your LLM workflow so you can see where things are working. 

Tracing helps you visualize the execution of your prompts and clearly see what inputs are being passed to the LLM. 

We'll use tracing to also get a better understanding of what's happening in our workflow by calling the `test_create_article` function that we import from the `orchestrator.py` file. We'll pass through the usual instructions for research and products the agents should get, and some context for what type of article should be written. `test_create_article` will then run all the logic necessary to generate an article. We also set local_tracing to true so that tracing is done locally. 

Once you can see the article has been generated, a `.runs` folder should appear in the `workshop` folder. Select this folder and click the `.tracy` file in it. 
This shows you all the Python functions that were called in order to generate the article. Explore each section and see what helpful information you can find that you might use in debugging! 

In [ ]:
# Add the path to sys.path
sys.path.append(os.path.abspath('../../src/api'))

from orchestrator import test_create_article
from tracing import init_tracing

tracer = init_tracing(local_tracing=True)

research_context = "Can you find the latest camping trends and what folks are doing in the winter?"
product_context = "Can you use a selection of tents and sleeping bags as context?"
assignment_context = '''Write a fun and engaging article that includes the research and product information. 
The article should be between 800 and 1000 words.
Make sure to cite sources in the article as you mention the research not at the end.'''

test_create_article(research_context=research_context, product_context=product_context, assignment_context=assignment_context)

Tracing is useful for deugging and observability both locally and in production. 

Let's now move on to the next section, building and running the full Contoso Creative Writer Application!

## 3. Building and running Contoso Creative Writer 

Now that we understand how the application works it's time to build it. 

To complete these next two learning outcomes you'll need to use the terminal. 

If it’s not already visible, you can open it by clicking on the hamburger menu at the top left of the page, clicking Terminal and then selecting New Terminal.

Once your terminal is open, copy and past the following commands in the terminal and press enter after each one to run it. 

1. Starting the FastAPI server 

    Navigate to the src/api folder  with the following command 

    ```shell
    cd ./src/api
    ```

    Run the FastAPI webserver with the following command 

    ```shell
    fastapi dev main.py
    ```

    Do not click open browser if prompted. 

    Next you'll need to change the visibility of the API's 8000 and 5173 ports to public in the `PORTS` tab. You can do this by right clicking on the visibility section of the port, selecting port visibility and setting it to public. The ports tab should look like this:

    <img src="../../images/ports.png" alt="Screenshot showing setting port-visibility" width="800px" />


2. Running the web app 

    Once you've completed the above steps. You'll need to open a **new terminal** and navigate to the web folder. you can open a new terminal by clicking on the hamburger menu at the top left of the page, clicking Terminal and then selecting New Terminal. 

    In the terminal run the following commands 

    ```shell
    cd ./src/web
    ```
    
    First install node packages:

    ```shell
    npm install
    ```

    Then run the web app with a local dev web server:
    
    ```shell
    npm run dev
    ```

    Once you've run the above command you should see an `http://localhost:5173/` link in the terminal. Click this link or click the `open on browser` button that comes up as a Gitub notification in the bottom right corner of the screen. Select the `continue` button and you should now see the app appear on your screen!

You can now test out the app by clicking `Example` to fill out the example information and then clicking `Start Work` to get Contoso Creative Writer to generate an article. 

You can also see which agent steps are being carried out in what order by click on the small bug button at the bottom right of the Application. The application with the generated article should look like this: 

<img src="../../images/agent.png" alt="Files in researcher agent" width="900" height="500">


## 4. Setting up automated evaluations and deployment with Github Actions 

Contoso Creative Writer is set up to run a CI/CD pipeline, which stands for Continuous Integration and Continuous Deployment. This is a series of automated steps that streamline the process of delivering software.

In this sample code the CI/CD pipeline includes the following: 
1. **Build and Deploy:** Automatically building and deploying the latest version of the code to production (This helps us confirm things are working as expected.)
2. **Evaulations:** Automatically sending example research, product and assignment instructions to Contoso Creative Writer and running evaulations on the produced article to see how fluent, grounded, relevant and x the final response was given the questions. 


To set up CI/CD with GitHub actions on your repository, **open a new terminal** and: 

1. Run the following command:

    ```shell
    azd pipeline config
    ```

    - select an environment name like yourname-aitour
    - select the recommended subscription by pressing enter
    - select `Canada East` as the Azure Location 
    - When asked to Log in using the Github CLI type in `Y`
    - Choose `HTTPS` as the preferred protocol for Git Operations 
    - Select `Y` to Authenticate with your Github credentials. 
    - Choose Login with a web browser to authenticate Github CLI and follow the authentication instructions. 
    - You may be asked if you want to commit and push your local changes. Choose `n`
    - You should see two links in your terminal. Select the Link to view your pipeline status. 

2. You should now be on a page that shows all the workflows. It should look similar to the image below. 

- Click on the workflow named evaluate (outlined in red in the image).
- It may need a few minutes to complete but once complete you should see the evaluated results on the page. 

You should see a table with some scores for relevance, fluencey, coherence and groundedness. The scores are from 1-5, with 5 being the highest mark. These are used to help us know how well the model is performing. This is helpful as a metric and 